In [1]:
import pandapower.networks as pn
import os
import pandapower as pp
import polars as pl
from polars import col as c
from shapely import  from_geojson
from plotly.graph_objects import Figure
import plotly.graph_objs as go
from general_function import pl_to_dict, build_non_existing_dirs

from data_display.grid_plotting import plot_grid_from_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from polars_function import list_to_list_of_tuple


os.chdir(os.getcwd().replace("/src", ""))


In [2]:
net = pn.case118()

2025-06-19 15:05:53 B01BPC15 pandapower.convert_format[105060] INFO These dtypes could not be corrected: {'trafo': ['tap_neutral', 'tap_min', 'tap_max', 'tap_pos']}


In [3]:
# net["sgen"] = net.gen[["name", "bus", "p_mw", "sn_mva", "scaling", "in_service", "type"]]
# net["sgen"]["q_mvar"] = 0
# net["sgen"]["current_source"] = True
net["gen"] = net["gen"].drop(net["gen"].index)
# remove all shunts
net.shunt = net.shunt.iloc[0:0]
# remove all polynomial cost entries
net.poly_cost = net.poly_cost.iloc[0:0]

In [4]:

bus: pl.DataFrame = pl.from_pandas(net["bus"]).with_columns(
    c("geo").map_elements(lambda x: list(from_geojson(x).coords)[0], return_dtype=pl.List(pl.Float64)),
).with_row_index(name="bus_id")


line: pl.DataFrame = pl.from_pandas(net["line"])
line = line.drop("name").with_row_index(name="name")\
.with_columns(
    ("Branche_" + c("name").cast(pl.Utf8)).alias("name")
).join(
    bus.select("bus_id", "vn_kv"),
    left_on="from_bus",
    right_on="bus_id",
    how="left"
)
trafo: pl.DataFrame = pl.from_pandas(net["trafo"]).drop("name").with_row_index(name="name").with_columns(
    ("Trafo_" + c("name").cast(pl.Utf8)).alias("name")
)

line

name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,geo,vn_kv
str,str,u32,u32,f64,f64,f64,f64,f64,f64,f64,u32,str,bool,f64,f64,f64
"""Branche_0""",null,0,1,1.0,5.770332,19.024956,353.789081,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_1""",null,0,2,1.0,2.456676,8.074656,150.708577,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_2""",null,3,4,1.0,0.3351744,1.5197112,29.250278,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_3""",null,2,4,1.0,4.589604,20.56752,395.575193,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_4""",null,4,5,1.0,2.266236,10.28376,198.623319,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Branche_168""",null,26,114,1.0,3.123216,14.111604,274.674042,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_169""",null,113,114,1.0,0.438012,1.980576,38.443223,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0
"""Branche_170""",null,11,116,1.0,6.265476,26.6616,498.647602,0.0,41.418606,1.0,1,"""ol""",true,100.0,null,138.0


In [5]:

switch = line.rename({"name": "line_name"}).with_row_index(name="name")\
.select(
    c("line_name"),
    "vn_kv",
    c("to_bus").alias("element"),
    pl.lit("b").alias("et"),
    pl.lit("LBS").alias("type"),
    pl.lit(True).alias("closed"),
    c("name"),
    pl.lit(0.0).alias("z_ohm"),
    pl.lit(100.0).alias("in_ka"),
).with_row_index(name="bus", offset=bus.height)\
.with_columns(
    ("switch " + c("name").cast(pl.Utf8)).alias("name"),
)
new_bus_mapping = pl_to_dict(switch["line_name", "bus"])

line = line.with_columns(
    c("name").replace_strict(new_bus_mapping, default=None).alias("to_bus")
)

bus = pl.concat([
    bus,
    switch.select(
    c("bus").alias("bus_id"),
    ("Bus " + c("bus").cast(pl.Utf8)).alias("name"),
    c("vn_kv"),
    pl.lit("b").alias("type"),
    pl.lit("1.0").alias("zone"),
    pl.lit(True).alias("in_service"),
)
], how="diagonal_relaxed")



net["bus"] = bus.to_pandas()
net["switch"] = switch.drop(["line_name", "vn_kv"]).to_pandas()
net["line"] = line.drop("vn_kv").to_pandas()
net["trafo"] = trafo.to_pandas()
net["ext_grid"]["vm_pu"] = 1.0
net["trafo"]["shift_degree"] = 0


build_non_existing_dirs(".cache/input_data")
pp.to_pickle(net, ".cache/input_data/mv_example_118_node.p")

In [6]:
pp.set_isolated_areas_out_of_service(net)

2025-06-19 15:05:53 B01BPC15 pandapower.toolbox.grid_modification[105060] INFO set 0 of 0 unsupplied buses out of service


In [7]:
net["switch"]

,bus,element,et,type,closed,name,z_ohm,in_ka
0,118,1,b,LBS,True,switch 0,0.0,100.0
1,119,2,b,LBS,True,switch 1,0.0,100.0
2,120,4,b,LBS,True,switch 2,0.0,100.0
3,121,4,b,LBS,True,switch 3,0.0,100.0
4,122,5,b,LBS,True,switch 4,0.0,100.0
...,...,...,...,...,...,...,...,...
168,286,114,b,LBS,True,switch 168,0.0,100.0
169,287,114,b,LBS,True,switch 169,0.0,100.0
170,288,116,b,LBS,True,switch 170,0.0,100.0
171,289,117,b,LBS,True,switch 171,0.0,100.0
